In [1]:
from retinavision_cython.retina import *
import skimage.io
import cv2
import time
import numpy as np
import sys
from os.path import dirname, join
import retinavision

from retinavision.cortex import Cortex
from retinavision import datadir, utils
from os.path import join
np.set_printoptions(threshold=sys.maxsize)
py = sys.version_info.major

if py == 2: import cPickle as pickle
elif py == 3: 
    import torch
    import pickle

# datadir = join(dirname(dirname(__file__)), "cythonised_retina")

def loadPickle(path):
        with open(path, 'rb') as handle:
            if py == 3: 
                return pickle.load(handle, encoding='latin1')
            return pickle.load(handle)

In [5]:
coeff = loadPickle(join("retinavision_cython", "data", "retinas", "ret50k_coeff.pkl"))

for i in range(0, 2):
    c = coeff[0,i]
    kernel = np.dstack((c,c,c))
    print(kernel.ndim)

3
3


In [2]:
img = skimage.io.imread('dock.jpg')
img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
fixation = (360, 640)

# coeff = 0 
# coeff_padded = 0
# loc = 0

In [3]:
R = retina_sample.Retina()
retina_sample.loadCoeff()
retina_sample.loadLoc()
R.updateLoc()
# img = img_gray.astype(np.int64)

In [5]:
%%timeit
R.sample(img_gray, fixation)

28.3 ms ± 754 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [2]:
cap = utils.camopen() #cap is the capture object (global)
ret, campic = cap.read()
# datadir = join(dirname(dirname(__file__)), "cythonised_retina")

#Create and load retina
R = retinavision.Retina()
R.info()
R.loadLoc(join("retinavision_cython", "data", "retinas", "ret50k_loc.pkl"))
R.loadCoeff(join("retinavision_cython", "data", "retinas", "ret50k_coeff.pkl"))

#Prepare retina
x = campic.shape[1]/2
y = campic.shape[0]/2
fixation = (y,x)
R.prepare(campic.shape, fixation)

while True:
    ret, img = cap.read()
    if ret is True:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img_ones = np.ones(img.shape)
        V = R.sample(img_ones, fixation)
        tight = R.backproject_tight_last()
        
        print(V[0])
        print(tight[0][0])
        break

loc - an Nx7 array containing retinal nodes defined as follows:
    [x, y, d, angle (radians), dist_5, rf_sigma, rf_width]
coeff - an array of variable size gaussian receptive field kernels
V - the imagevector, output of retinal sampling
gaussNorm - Gaussian normalization image for producing backprojections

REMEMBER: all coordinates are tuples in the Y,X order, not X,Y.
The only exception is the loc array
REMEMBER2: coeff is redundantly wrapped in another matrix for backwards compatibility
1.0
0


In [5]:
tight = R.backproject_tight_last()

print(tight.shape)

IndexError: Out of bounds on buffer access (axis 0)